In [1]:
# -*- coding: utf-8 -*-


import json
import cchardet as chardet
from pprint import pprint
from tqdm import tnrange, tqdm_notebook
from time import sleep
from bs4 import BeautifulSoup
from pymongo import MongoClient
from pymongo.mongo_client import database
from pymongo.collection import Collection
import pandas as pd
from pprint import pprint
from pymongo import IndexModel, ASCENDING, DESCENDING


def tqdm_ipython_test():
    for i in tnrange(3, desc='1st loop'):
        for j in tqdm_notebook(range(100), desc='2nd loop'):
            sleep(0.01)


def read_osm_file(filename: str):
    with open(filename, "r", encoding='UTF-8') as f:
        msg = f.read()
        # result = chardet.detect(msg)
    return msg


def get_soup(file, tags):
    soup = BeautifulSoup(file, 'xml')
    return [{tag: soup.find_all(tag)} for tag in tqdm_notebook(tags)]


def get_dict_data(result_set_item):
    list_of_dicts = []
    for k, v in result_set_item:
        primary_tag = k
        result_set = v
    for entry in result_set:
        entry_data_dict = {}
        entry_data_dict['type'] = primary_tag
        for k, v in entry.attrs.items():
            entry_data_dict[k] = v
        for tag in entry.find_all('tag'):
            entry_data_dict[tag['k']] = tag['v']
        list_of_dicts.append(entry_data_dict)
    return list_of_dicts


def json_to_mongo(col: database.Collection, json_file: str ="rochester_osm.json" ):
    # data = []
    with open(json_file) as f:
        data = json.load(f)
        #for line in f:
        #    data.append(json.loads(line))
    for node_dict in data:
        col.insert_many(node_dict)
    # return col.insert_many(data)
    


def get_col(db_name="udacity", collection="rochester_osm"):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    col = db[collection]
    return col


def main():
    file_name = "rochester_ny.osm"
    osm_file = read_osm_file(filename=file_name)
    tag_list = ['node', 'way']
    result_set_list = get_soup(file=osm_file, tags=tag_list)
    osm_dicts = [get_dict_data(res) for res in result_set_list.values()]
    with open("osm_dicts", 'w') as f:
        f.write(osm_dicts)

### Code to convert the osm to json in preparation for insertion to MongoDB
```python
# Loading The osm file
file_name = r"rochester_ny.osm"
osm_file = read_osm_file(filename=file_name)
# Loading the osm file into beautiful soup and grabbing all node and way tags
tag_list = ['node', 'way']
%time result_set_list = get_soup(file=osm_file, tags=tag_list)
# make list of dictionaries containing the attribute and tag data for the result set
osm_dicts = [get_dict_data(res.items()) for res in result_set_list]
# osm_dicts = [get_dict_data(res) for res in result_set_list.values()]
# dump this dict data to a json object so that parsing doesn't need to be re-run
json_osm = json.dumps(osm_dicts)
# write the json to file
with open('rochester_osm.json', 'w') as f:
    f.write(json_osm)
```

### Initial MongoDB collection creation
- Insert all records from json file
- Create compound unique index on 'id' and 'type' fields

In [2]:
# read the json file we just read to verify it's working
from importlib import import_module
j2m = import_module('json_to_mongo')
%time j2m.main()

457947 records inserted from rochester_osm.json
[{'key': SON([('_id', 1)]),
  'name': '_id_',
  'ns': 'udacity.rochester_osm',
  'v': 2},
 {'key': SON([('id', 1), ('type', -1)]),
  'name': 'id_type_unique_index',
  'ns': 'udacity.rochester_osm',
  'unique': True,
  'v': 2}]
Wall time: 14.8 s


In [3]:
# setup connection for data exploration and cleaning
osm_col = get_col() # type: MongoClient

### Query total document count

In [4]:
total_docs = osm_col.count_documents({})
total_docs

457947

### Get count of each key in collection

In [5]:
key_counts_dict = dict()
for entry in tqdm_notebook(osm_col.find(), total=total_docs):
    for k in entry.keys():
        key_counts_dict.setdefault(k, 0)
        key_counts_dict[k] += 1

In [33]:
[print(f"{k}, {v} ") for k,v in key_counts_dict.items() if ':city' in k]

addr:city, 5104 
is_in:city, 1 
addr:city_1, 2 


[None, None, None]

In [6]:
#itemgetter used with sorted to allow sorting by key values
from operator import itemgetter
pprint(sorted(key_counts_dict.items(), key=itemgetter(1), reverse=True))

[('_id', 457947),
 ('type', 457947),
 ('id', 457947),
 ('version', 457947),
 ('timestamp', 457947),
 ('changeset', 457947),
 ('uid', 457947),
 ('user', 457947),
 ('lat', 405420),
 ('lon', 405420),
 ('highway', 32238),
 ('name', 17997),
 ('building', 14960),
 ('tiger:county', 12249),
 ('tiger:cfcc', 12242),
 ('tiger:name_base', 11459),
 ('tiger:name_type', 9816),
 ('tiger:zip_left', 8696),
 ('tiger:zip_right', 8478),
 ('tiger:reviewed', 8253),
 ('service', 6392),
 ('addr:street', 5587),
 ('addr:housenumber', 5238),
 ('addr:postcode', 5123),
 ('addr:city', 5104),
 ('addr:state', 4612),
 ('surface', 3766),
 ('amenity', 3739),
 ('oneway', 3347),
 ('power', 2814),
 ('source', 2228),
 ('access', 2052),
 ('addr:country', 1994),
 ('ref', 1887),
 ('footway', 1880),
 ('leisure', 1528),
 ('lanes', 1439),
 ('landuse', 1400),
 ('railway', 1242),
 ('tiger:source', 1225),
 ('tiger:upload_uuid', 1224),
 ('tiger:name_base_1', 1217),
 ('tiger:tlid', 1211),
 ('operator', 1021),
 ('hgv', 1005),
 ('parking

### Get a list of fields that begin with address

In [7]:

address_fields = {k:v for (k, v) in key_counts_dict.items() if 'addr' in k}
pprint(sorted(address_fields.items(), key=itemgetter(1), reverse=True))



[('addr:street', 5587),
 ('addr:housenumber', 5238),
 ('addr:postcode', 5123),
 ('addr:city', 5104),
 ('addr:state', 4612),
 ('addr:country', 1994),
 ('addr:unit', 68),
 ('addr:housenumber_1', 34),
 ('addr:housename', 33),
 ('addr:housenumber_2', 5),
 ('addr:housenumber_3', 5),
 ('addr:housenumber_4', 5),
 ('addr:housenumber_5', 5),
 ('addr:street_1', 4),
 ('addr:floor', 2),
 ('addr:place', 2),
 ('addr:city_1', 2),
 ('addr:street_2', 2),
 ('addr:province', 1),
 ('addr:floot', 1),
 ('addr:pobox', 1),
 ('addr:full', 1),
 ('addr:street_3', 1),
 ('addr:housenumber_6', 1),
 ('addr:housenumber_7', 1),
 ('source:addr', 1)]


In [8]:
# Get a list of distinct streets
distinct_streets = osm_col.distinct('addr:street')

In [9]:
# Get a list of distinct street types
pprint(set(x.split()[-1] for x in distinct_streets))

{'#102',
 '#2',
 '#A-2',
 '31',
 '92',
 'Apartment',
 'Ave',
 'Ave.',
 'Avenu',
 'Avenue',
 'Bend',
 'Blvd',
 'Boulelvard',
 'Boulevard',
 'Bridge',
 'Center',
 'Cir',
 'Circle',
 'Court',
 'Crescent',
 'Ct',
 'Dr',
 'Drive',
 'Drop',
 'East',
 'Green',
 'Highway',
 'Hill',
 'Homes',
 'Landing',
 'Lane',
 'Manor',
 'Market',
 'Meadows',
 'N',
 'North',
 'Oaks',
 'PW',
 'Park',
 'Parkway',
 'Passage',
 'Place',
 'Race',
 'Rd',
 'Rd.',
 'Rise',
 'Road',
 'Run',
 'S',
 'South',
 'Spruce',
 'Square',
 'St',
 'St.',
 'Stree',
 'Street',
 'Trail',
 'Villas',
 'W',
 'Way',
 'West',
 'Woods',
 'ave',
 'line'}


### find all address codes in collection

In [10]:
unique_zip_codes = osm_col.distinct('addr:postcode')
pprint(unique_zip_codes)

['14607',
 '14624',
 '14617',
 '14623',
 '14622',
 '14612',
 '14626',
 '14450',
 '14618',
 '14616',
 '14526',
 '14502',
 '14514',
 '14615',
 '14580',
 '14620',
 '14625',
 '14445',
 '14608',
 '14609',
 '14606',
 '14559',
 '14621',
 '14613',
 '14534',
 '14604',
 '14614',
 '14620-1327',
 'West Main Street',
 '14694',
 '14605',
 '14610',
 '14611',
 '14468',
 '14607-2082',
 '14519',
 '14642',
 '14627',
 '14624-4721',
 '14617-1822',
 '14467',
 '14692',
 '14568',
 '14543',
 '14586',
 '14428',
 '1445033',
 '14424',
 '14619']


In [11]:
update_dict = {'modified': 0,
              'deleted': 0,
              'good': 0}
for zip in tqdm_notebook(unique_zip_codes):
    if zip[0:5].isdigit() and len(zip) > 5:
        result = osm_col.update_many({'addr:postcode': zip}, {"$set": {'addr:postcode': zip[0:5]}}) 
        update_dict['modified'] += result.modified_count
    elif not zip.isdigit() and len(zip)!=5:
        result = osm_col.delete_many({'addr:postcode': zip})
        update_dict['deleted'] += result.deleted_count
    elif zip.isdigit() and len(zip)==5:
        update_dict['good'] += 1

pprint(update_dict)


{'deleted': 1, 'good': 43, 'modified': 6}


In [12]:
#
updated_address_code_list = list(osm_col.find({'addr:postcode': {'$exists': True}}, {'addr:postcode': 1, '_id': 0}))
set([x['addr:postcode'] for x in updated_address_code_list])


{'14424',
 '14428',
 '14445',
 '14450',
 '14467',
 '14468',
 '14502',
 '14514',
 '14519',
 '14526',
 '14534',
 '14543',
 '14559',
 '14568',
 '14580',
 '14586',
 '14604',
 '14605',
 '14606',
 '14607',
 '14608',
 '14609',
 '14610',
 '14611',
 '14612',
 '14613',
 '14614',
 '14615',
 '14616',
 '14617',
 '14618',
 '14619',
 '14620',
 '14621',
 '14622',
 '14623',
 '14624',
 '14625',
 '14626',
 '14627',
 '14642',
 '14692',
 '14694'}

[Rochester Zip codes](https://www.zip-codes.com/city/ny-rochester.asp)
> After running our function we can see that all the unique zip codes in the database are valid Rochester Zip codes

In [70]:
pprint(list(osm_col.aggregate([
    {
        '$group': {
            '_id': '$addr:postcode', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]))[:10])

[{'_id': None, 'count': 452824},
 {'_id': '14450', 'count': 1624},
 {'_id': '14624', 'count': 445},
 {'_id': '14618', 'count': 421},
 {'_id': '14623', 'count': 392},
 {'_id': '14534', 'count': 346},
 {'_id': '14626', 'count': 300},
 {'_id': '14514', 'count': 264},
 {'_id': '14612', 'count': 226},
 {'_id': '14620', 'count': 177}]


# User Counts

In [13]:
def get_single_users(col: Collection):
    user_counts_dict = list(col.aggregate([
        {
            '$sortByCount': '$user'
        }, {
            '$sort': {
                'count': 1
            }
        }
    ]))
    single_doc_user = list()
    for entry in user_counts_dict:
        if entry['count'] == 1:
           single_doc_user.append(entry['_id'])
        else:
            break
    pprint(single_doc_user[0:5])
    pprint(f"{len(single_doc_user)} users with only one post out of {len(user_counts_dict)}")    
    # return single_doc_user
get_single_users(osm_col)


['dgitto', 'Takuto', 'lonvia', 'glglgl', 'ayazhaider9']
'146 users with only one post out of 719'


In [14]:
user_df = pd.DataFrame.from_dict(list(osm_col.aggregate([ {
            '$sortByCount': '$user'
        }])))
user_df['percent']= user_df['count']/user_df['count'].sum()

In [15]:
# Percent of entries that came from top two users
user_df[0:2]['percent'].sum()*100

23.01821612155145

In [16]:
# Combined top 10 users contribution
user_df[0:10]['percent'].sum()*100

61.08449467841187

In [17]:
# Combined perecent of users who individually contribute less then 1% of the entries in the database
user_df[user_df.percent <= .01].percent.sum()*100

24.901407589541126

In [18]:
user_df._id

0        woodpeck_fixbot
1                 wambag
2               dankpoet
3               idrive66
4                 ECRock
5                 sivart
6             RussNelson
7                   timr
8                  T-Rex
9        visionsofkenobi
10      URcommunications
11                 stuuf
12        Craig Williams
13             paperboat
14           devrintalen
15                gadget
16                  fx99
17                ColumM
18              bot-mode
19        Nathan Willard
20                 Hooka
21                McColl
22                canisd
23                deejoe
24                Hwyfan
25             jwernerny
26              kbzimmer
27            slugmuffin
28             njtbusfan
29               jsb2092
             ...        
689                hakan
690            Mickael S
691             dmouhama
692             Bhojaraj
693              Anthony
694                 SK53
695        JustinColeGIS
696    Marcussacapuces91
697          Wendy Marks


In [19]:
def top_ten_amenities(col: Collection):
    top_amenities= list(col.aggregate([
    {
        '$match': {
            'type': 'way'
        }
    }, {
        '$sortByCount': '$amenity'
    }
]))
    return top_amenities
top_ten_amenities(col=osm_col)[0:10] 
   

[{'_id': None, 'count': 49953},
 {'_id': 'parking', 'count': 1828},
 {'_id': 'restaurant', 'count': 127},
 {'_id': 'school', 'count': 85},
 {'_id': 'fuel', 'count': 56},
 {'_id': 'fast_food', 'count': 54},
 {'_id': 'place_of_worship', 'count': 49},
 {'_id': 'bank', 'count': 46},
 {'_id': 'shelter', 'count': 43},
 {'_id': 'fire_station', 'count': 32}]

In [20]:



df = pd.DataFrame.from_dict(top_ten_amenities(osm_col))
pprint(df[0:10])

                _id  count
0              None  49953
1           parking   1828
2        restaurant    127
3            school     85
4              fuel     56
5         fast_food     54
6  place_of_worship     49
7              bank     46
8           shelter     43
9      fire_station     32


In [21]:
df['percent'] = df['count']/df['count'].sum()

``` table 
_id|count|percent\r\n|49953|0.951069055461417\r\nparking|1828|0.034803800239894905\r\nrestaurant|127|0.0024179883098832894\r\nschool|85|0.001618338632599052\r\nfuel|56|0.0010661995697123165\r\n
```

In [22]:
df[1:].describe()

,count,percent
count,48.000000,48.000000
mean,53.541667,0.001019
std,262.708232,0.005002
min,1.000000,0.000019
25%,3.000000,0.000057
50%,6.500000,0.000124
75%,14.250000,0.000271
max,1828.000000,0.034804


In [23]:
df.shape[0]

49

In [25]:
# Biggest Religion
religion = list(osm_col.aggregate([
    {
        '$match': {
            'amenity': {
                '$eq': 'place_of_worship'
            }
        }
    }, {
        '$group': {
            '_id': '$religion', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]))
pprint(religion)

[{'_id': 'christian', 'count': 119},
 {'_id': None, 'count': 19},
 {'_id': 'jewish', 'count': 2},
 {'_id': 'buddhist', 'count': 1},
 {'_id': 'muslim', 'count': 1}]


In [27]:
# Most popular cuisine in restaurants
cuisine = list(osm_col.aggregate([
    {
        '$match': {
            'amenity': {
                '$eq': 'restaurant'
            }
        }
    }, {
        '$group': {
            '_id': '$cuisine', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]))
pprint(cuisine[0:10])

[{'_id': None, 'count': 119},
 {'_id': 'pizza', 'count': 31},
 {'_id': 'american', 'count': 25},
 {'_id': 'italian', 'count': 15},
 {'_id': 'burger', 'count': 14},
 {'_id': 'sandwich', 'count': 8},
 {'_id': 'mexican', 'count': 8},
 {'_id': 'chinese', 'count': 7},
 {'_id': 'japanese', 'count': 5},
 {'_id': 'thai', 'count': 4}]


In [67]:
#City counts
city_counts = list(osm_col.aggregate([
    {
        '$group': {
            '_id': '$addr:city', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]))
print('\n'.join('{_id!s:<20}{count}'.format(**x) for x in city_counts))

None                452842
Rochester           2227
Fairport            1612
Pittsford           350
North Chili         264
Brighton            205
Greece              161
Webster             86
Henrietta           35
Churchville         29
West Henrietta      18
East Rochester      17
Rochester, NY       16
Penfield            16
Perinton            15
Hilton              15
Spencerport         11
North Greece        5
Macedon             5
Riga                2
rochester           2
Ontario, NY         2
Walworth            2
Gates               2
Irondequoit         2
Ontario             1
W Commercial St     1
Rochestet           1
pittsford           1
East Rochester Town 1


In [44]:
correct = 'correct'
phonetic_correct = 'phonetic_correct'
typo = 'typo'
phonetic_typo = 'phonetic_typo'
phonetic_distance = 'phonetic_distance'

print(f'No Spacing:')
print(f'{correct}|{phonetic_correct}|{typo}|{phonetic_typo}|{phonetic_distance}|\n')
# No Spacing:
# correct|phonetic_correct|typo|phonetic_typo|phonetic_distance|

print(f'Right Aligned:')
print(f'{correct:>10}|{phonetic_correct:>20}|{typo:>10}|{phonetic_typo:>20}|{phonetic_distance:>20}|\n')
# Right Aligned:

No Spacing:
correct|phonetic_correct|typo|phonetic_typo|phonetic_distance|

Right Aligned:
   correct|    phonetic_correct|      typo|       phonetic_typo|   phonetic_distance|



| Command | Description |
| --- | --- |
| `git status` | List all *new or modified* files |
| `git diff` | Show file differences that **haven't been** staged |